In [54]:
import pandas as pd
import json
import requests

# read csv data on counties, economics

In [327]:
black_income = pd.read_csv('ACS_16_5YR_B19001B_with_ann.csv', encoding = "ISO-8859-1", skiprows = 1)

In [328]:
black_income['County'], black_income['State'] = zip(*black_income['Geography'].apply(lambda x: x.split(',', 1)))

In [330]:
black_income.set_index('Id2', drop = False, inplace = True)

In [331]:
black_income.shape

(3142, 39)

# read bostock tsv data on unemployment

In [332]:
unemployment = pd.read_csv('unemployment.tsv', sep = '\t')

In [333]:
unemployment.set_index('id', drop = False, inplace = True)

In [334]:
unemployment.shape

(3219, 2)

# merge unemployment and black income

In [335]:
merged_csv = pd.merge(black_income, unemployment, left_index = True, right_index = True)

In [336]:
merged_csv.shape

(3141, 41)

In [337]:
merged_csv['State'] = merged_csv['State'].str.strip()

In [338]:
merged_csv['id'] = merged_csv['id'].astype('str')

In [339]:
merged_csv['id'] = merged_csv['id'].str.zfill(5)

In [340]:
merged_csv = merged_csv[merged_csv['Estimate; Total:']> 0 ]

In [341]:
merged_csv['percent_poor'] = (
            (merged_csv['Estimate; Total: - Less than $10,000']
                    + merged_csv['Estimate; Total: - $10,000 to $14,999']
                    + merged_csv['Estimate; Total: - $15,000 to $19,999'])
                    / merged_csv['Estimate; Total:']
            )
merged_csv['percent_poor'] = merged_csv['percent_poor'].round(2)
merged_csv['percent_rich'] = merged_csv['Estimate; Total: - $200,000 or more'] / merged_csv['Estimate; Total:']

In [342]:
pd.set_option('precision', 2)

In [343]:
merged_csv.to_csv('income_unemployment.csv')

In [344]:
merged_csv.shape

(2716, 43)

In [266]:
x = merged_csv[merged_csv['Id2'] == 2068]

In [267]:
(x['Margin of Error; Total: - Less than $10,000']
                    + x['Estimate; Total: - $10,000 to $14,999']
                    + x['Estimate; Total: - $15,000 to $19,999'])

2068    14
dtype: int64

In [270]:
x['Margin of Error; Total: - Less than $10,000']

2068    9
Name: Margin of Error; Total: - Less than $10,000, dtype: int64

# read geojson - spatial shape data on counties

In [75]:
import requests

In [126]:
x = requests.get('https://d3js.org/us-10m.v1.json')

In [127]:
r = x.json()

In [128]:
r.keys()

dict_keys(['bbox', 'arcs', 'objects', 'transform', 'type'])

In [79]:
r['objects'].keys()

dict_keys(['nation', 'states', 'counties'])

In [110]:
r['objects']['counties']['geometries']

[{'arcs': [[0, 1, 2, 3, 4]], 'id': '05089', 'type': 'Polygon'},
 {'arcs': [[5, 6, 7, 8, 9]], 'id': '06079', 'type': 'Polygon'},
 {'arcs': [[10, 11, 12, 13, 14, 15, 16]], 'id': '17111', 'type': 'Polygon'},
 {'arcs': [[17, 18, 19, 20, 21, 22]], 'id': '20131', 'type': 'Polygon'},
 {'arcs': [[23, 24, 25, 26, 27, 28, 29]], 'id': '28141', 'type': 'Polygon'},
 {'arcs': [[30, 31, 32, 33, 34, 35, 36]], 'id': '39009', 'type': 'Polygon'},
 {'arcs': [[37, 38, 39, 40, 41]], 'id': '48369', 'type': 'Polygon'},
 {'arcs': [[42, 43, 44, 45, 46]], 'id': '48211', 'type': 'Polygon'},
 {'arcs': [[47, 48, 49, 50, 51, 52]], 'id': '01007', 'type': 'Polygon'},
 {'arcs': [[53, 54, 55, 56, 57]], 'id': '05111', 'type': 'Polygon'},
 {'arcs': [[58, 59, 60, 61, 62, 63]], 'id': '19105', 'type': 'Polygon'},
 {'arcs': [[64, 65, 66, 67, 68, 69]], 'id': '20111', 'type': 'Polygon'},
 {'arcs': [[70, 71, 72, 73, 74, 75]], 'id': '20171', 'type': 'Polygon'},
 {'arcs': [[76, 77, 78, 79, 80]], 'id': '22003', 'type': 'Polygon'},


# convert json to csv for merging

In [80]:
from pandas.io.json import json_normalize
csved = json_normalize(r['objects']['counties']['geometries'])

In [81]:
# !!!!!!!!!
# this got rid of some leading zeros...could be a problem here
# !!!!!!!!!
csved['id'] = pd.to_numeric(csved['id'])

In [82]:
csved.set_index('id', drop = False, inplace = True)

# merge csv, json

In [83]:
# for county in r['objects']['counties']['geometries']:
#     if county['id'] == '05089':
#         print( county['id'] )

In [92]:
merged = pd.merge(black_county_income, csved, left_index = True, right_index = True)

In [93]:
merged.head()

,"Estimate; Total: - Less than $10,000","Estimate; Total: - $200,000 or more",arcs,id,type
1001,488,28,"[[-6749, -6744, -4943, 8740, -4945]]",1001,Polygon
1003,787,24,"[[143, 144, 145, 146, 147, 148, 149]]",1003,Polygon
1005,926,8,"[[-6295, -4536, 8741, -8176, -826, 8742, -8646...",1005,Polygon
1007,154,0,"[[47, 48, 49, 50, 51, 52]]",1007,Polygon
1009,16,0,"[[-6209, -3873, 6210, -6189, -2101, -6195]]",1009,Polygon


# convert back to json!


In [117]:
merged.to_json('newJSON.json', orient = 'records')

# combine this json with original topo file format

In [118]:
with open('newJSON.json') as json_data:
    d = json.load(json_data)

In [120]:
r['objects']['counties']['geometries'] = d

In [121]:
r['objects']['counties']

{'geometries': [{'Estimate; Total: - $200,000 or more': 28,
   'Estimate; Total: - Less than $10,000': 488,
   'arcs': [[-6749, -6744, -4943, 8740, -4945]],
   'id': 1001,
   'type': 'Polygon'},
  {'Estimate; Total: - $200,000 or more': 24,
   'Estimate; Total: - Less than $10,000': 787,
   'arcs': [[143, 144, 145, 146, 147, 148, 149]],
   'id': 1003,
   'type': 'Polygon'},
  {'Estimate; Total: - $200,000 or more': 8,
   'Estimate; Total: - Less than $10,000': 926,
   'arcs': [[-6295, -4536, 8741, -8176, -826, 8742, -8646, 8743]],
   'id': 1005,
   'type': 'Polygon'},
  {'Estimate; Total: - $200,000 or more': 0,
   'Estimate; Total: - Less than $10,000': 154,
   'arcs': [[47, 48, 49, 50, 51, 52]],
   'id': 1007,
   'type': 'Polygon'},
  {'Estimate; Total: - $200,000 or more': 0,
   'Estimate; Total: - Less than $10,000': 16,
   'arcs': [[-6209, -3873, 6210, -6189, -2101, -6195]],
   'id': 1009,
   'type': 'Polygon'},
  {'Estimate; Total: - $200,000 or more': 2,
   'Estimate; Total: - L

In [122]:
with open('topo_and_economic_data.json', 'w') as outfile:
    json.dump(r, outfile)

In [123]:
with open('topo_and_economic_data.json') as json_data:
    check = json.load(json_data)

In [124]:
check.keys()

dict_keys(['bbox', 'type', 'objects', 'transform', 'arcs'])

In [129]:
r.keys()

dict_keys(['bbox', 'arcs', 'objects', 'transform', 'type'])

In [140]:
r['objects']['counties']['geometries']

[{'arcs': [[0, 1, 2, 3, 4]], 'id': '05089', 'type': 'Polygon'},
 {'arcs': [[5, 6, 7, 8, 9]], 'id': '06079', 'type': 'Polygon'},
 {'arcs': [[10, 11, 12, 13, 14, 15, 16]], 'id': '17111', 'type': 'Polygon'},
 {'arcs': [[17, 18, 19, 20, 21, 22]], 'id': '20131', 'type': 'Polygon'},
 {'arcs': [[23, 24, 25, 26, 27, 28, 29]], 'id': '28141', 'type': 'Polygon'},
 {'arcs': [[30, 31, 32, 33, 34, 35, 36]], 'id': '39009', 'type': 'Polygon'},
 {'arcs': [[37, 38, 39, 40, 41]], 'id': '48369', 'type': 'Polygon'},
 {'arcs': [[42, 43, 44, 45, 46]], 'id': '48211', 'type': 'Polygon'},
 {'arcs': [[47, 48, 49, 50, 51, 52]], 'id': '01007', 'type': 'Polygon'},
 {'arcs': [[53, 54, 55, 56, 57]], 'id': '05111', 'type': 'Polygon'},
 {'arcs': [[58, 59, 60, 61, 62, 63]], 'id': '19105', 'type': 'Polygon'},
 {'arcs': [[64, 65, 66, 67, 68, 69]], 'id': '20111', 'type': 'Polygon'},
 {'arcs': [[70, 71, 72, 73, 74, 75]], 'id': '20171', 'type': 'Polygon'},
 {'arcs': [[76, 77, 78, 79, 80]], 'id': '22003', 'type': 'Polygon'},


In [141]:
check['objects']['counties']['geometries']

[{'Estimate; Total: - $200,000 or more': 28,
  'Estimate; Total: - Less than $10,000': 488,
  'arcs': [[-6749, -6744, -4943, 8740, -4945]],
  'id': 1001,
  'type': 'Polygon'},
 {'Estimate; Total: - $200,000 or more': 24,
  'Estimate; Total: - Less than $10,000': 787,
  'arcs': [[143, 144, 145, 146, 147, 148, 149]],
  'id': 1003,
  'type': 'Polygon'},
 {'Estimate; Total: - $200,000 or more': 8,
  'Estimate; Total: - Less than $10,000': 926,
  'arcs': [[-6295, -4536, 8741, -8176, -826, 8742, -8646, 8743]],
  'id': 1005,
  'type': 'Polygon'},
 {'Estimate; Total: - $200,000 or more': 0,
  'Estimate; Total: - Less than $10,000': 154,
  'arcs': [[47, 48, 49, 50, 51, 52]],
  'id': 1007,
  'type': 'Polygon'},
 {'Estimate; Total: - $200,000 or more': 0,
  'Estimate; Total: - Less than $10,000': 16,
  'arcs': [[-6209, -3873, 6210, -6189, -2101, -6195]],
  'id': 1009,
  'type': 'Polygon'},
 {'Estimate; Total: - $200,000 or more': 2,
  'Estimate; Total: - Less than $10,000': 606,
  'arcs': [[6293,